meal recommendation


In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk


In [2]:
df_normal = pd.read_excel("NLP DATASET.xlsx")
df_diseased = pd.read_excel("nlp dieseases dataset.xlsx")


In [3]:
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'\W+', ' ', text.lower())
    words = text.split()
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]
    return ' '.join(words)


In [4]:
df_normal['features'] = (df_normal['type'] + " " + df_normal['meal type'] + " " + 
                         df_normal['person type'] + " " + df_normal['diet type'])
df_normal['features'] = df_normal['features'].apply(preprocess_text)

df_diseased['features'] = (df_diseased['type'] + " " + df_diseased['meal type'] + 
                           " " + df_diseased['disease type'])
df_diseased['features'] = df_diseased['features'].apply(preprocess_text)


Setup the Fine-Tuned NER Model

In [5]:
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
ner_pipeline = pipeline("ner", model=model_name, tokenizer=model_name, aggregation_strategy="simple")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Rule-Based Entity Extraction Function

In [6]:
def rule_based_entity_extraction(text):
    meal_types = ['breakfast', 'lunch', 'dinner']
    food_types = ['veg', 'non-veg']
    person_types = ['weight gain', 'weight loss', 'atheletic', 'normal']
    diet_types = ['low carb', 'high protein', 'normal']
    disease_types = ['diabetes', 'hypertension', 'obesity', 'heart disease']

    entities = {
        'meal_type': None,
        'food_type': None,
        'person_type': None,
        'diet_type': None,
        'disease_type': None
    }
    
    for meal in meal_types:
        if meal in text.lower():
            entities['meal_type'] = meal
            break
    
    for food in food_types:
        if food in text.lower():
            entities['food_type'] = food
            break
    
    for person in person_types:
        if person in text.lower():
            entities['person_type'] = person
            break
    
    for diet in diet_types:
        if diet in text.lower():
            entities['diet_type'] = diet
            break
    
    for disease in disease_types:
        if disease in text.lower():
            entities['disease_type'] = disease
            break
    
    return entities


Entity Extraction Combining BERT and Rule-Based Approaches

In [7]:
def extract_entities(text):
    bert_entities = ner_pipeline(text)
    rule_entities = rule_based_entity_extraction(text)
    
    entities = {
        'meal_type': rule_entities['meal_type'],
        'food_type': rule_entities['food_type'],
        'person_type': rule_entities['person_type'],
        'diet_type': rule_entities['diet_type'],
        'disease_type': rule_entities['disease_type'],
    }
    
    return entities


Load the Sentence Transformer Model

In [8]:
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


C:\Users\VIKRAM\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Function to Get Sentence Embedding


In [9]:
def get_sentence_embedding(text, model):
    return model.encode(text)


Vectorize Both Normal and Diseased Datasets

In [10]:
df_normal['embedding'] = df_normal['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))
df_diseased['embedding'] = df_diseased['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))


C:\Users\VIKRAM\AppData\Roaming\Python\Python311\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [11]:
df_normal['embedding'] = df_normal['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))
df_diseased['embedding'] = df_diseased['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))


Function to Recommend Food Based on Extracted Entities

In [12]:
def recommend_food_based_on_entities(entities):
    if entities['disease_type']:
        user_input = f"{entities['food_type']} {entities['meal_type']} {entities['disease_type']}"
        user_embedding = get_sentence_embedding(user_input, sentence_model)
        similarities = cosine_similarity([user_embedding], np.stack(df_diseased['embedding'].values))
        idx = np.argmax(similarities)
        return df_diseased['recommend'].iloc[idx]
    else:
        user_input = f"{entities['food_type']} {entities['meal_type']} {entities['person_type']} {entities['diet_type']}"
        user_embedding = get_sentence_embedding(user_input, sentence_model)
        similarities = cosine_similarity([user_embedding], np.stack(df_normal['embedding'].values))
        idx = np.argmax(similarities)
        return df_normal['recommend'].iloc[idx]


get recommendationn

In [13]:
def get_recommendation():
    prompt = input("Please describe your diet preferences: ")
    extracted_entities = extract_entities(prompt)
    print(f"Extracted Entities: {extracted_entities}")
    recommendation = recommend_food_based_on_entities(extracted_entities)
    print("Recommended Food:", recommendation)


get_recommendation()

Extracted Entities: {'meal_type': 'breakfast', 'food_type': 'veg', 'person_type': 'normal', 'diet_type': 'normal', 'disease_type': None}
Recommended Food: idly or dosa or any type of breakfast


Nutritional analysis

In [ ]:
import requests

# Set your Edamam API credentials
EDAMAM_APP_ID = '2a98c6a4'  # Replace with your Edamam APP ID
EDAMAM_APP_KEY = '75704984fc122dc3153ae7a943f3cb56'  # Replace with your Edamam APP KEY

# Function to get nutritional information from Edamam API
def get_nutritional_info(food_item):
    url = f"https://api.edamam.com/api/nutrition-data?app_id={EDAMAM_APP_ID}&app_key={EDAMAM_APP_KEY}&nutrition-type=logging&ingr={food_item}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to format nutritional information for output
def format_nutrition_info(nutrition_data):
    if nutrition_data:
        nutrients = nutrition_data.get('totalNutrients', {})
        formatted_info = []
        for nutrient, details in nutrients.items():
            formatted_info.append(f"{details['label']}: {details['quantity']} {details['unit']}")
        return "\n".join(formatted_info)
    return "Nutritional information not available."

# Function to get user input and perform nutritional analysis
def get_nutritional_analysis():
    # Get user input for food item
    food_item = input("Enter the food item you want nutritional information for: ")

    # Get nutritional analysis for the food item
    nutrition = get_nutritional_info(food_item)
    nutritional_analysis = format_nutrition_info(nutrition)
    print("Nutritional Analysis:\n", nutritional_analysis)

# Run the system
get_nutritional_analysis()


In [1]:
import numpy as np
import pandas as pd
import re
import requests
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk
import tkinter as tk
from tkinter import messagebox

# Load datasets
df_normal = pd.read_excel("NLP DATASET.xlsx")
df_diseased = pd.read_excel("nlp dieseases dataset.xlsx")

# Preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'\W+', ' ', text.lower())
    words = text.split()
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]
    return ' '.join(words)

# Prepare features
df_normal['features'] = (df_normal['type'] + " " + df_normal['meal type'] + " " + 
                         df_normal['person type'] + " " + df_normal['diet type'])
df_normal['features'] = df_normal['features'].apply(preprocess_text)

df_diseased['features'] = (df_diseased['type'] + " " + df_diseased['meal type'] + 
                           " " + df_diseased['disease type'])
df_diseased['features'] = df_diseased['features'].apply(preprocess_text)

# Load NER model
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
ner_pipeline = pipeline("ner", model=model_name, tokenizer=model_name, aggregation_strategy="simple")

# Rule-based entity extraction function
def rule_based_entity_extraction(text):
    meal_types = ['breakfast', 'lunch', 'dinner']
    food_types = ['veg', 'non-veg']
    person_types = ['weight gain', 'weight loss', 'athletic', 'normal']
    diet_types = ['low carb', 'high protein', 'normal']
    disease_types = ['diabetes', 'hypertension', 'obesity', 'heart disease']

    entities = {
        'meal_type': None,
        'food_type': None,
        'person_type': None,
        'diet_type': None,
        'disease_type': None
    }
    
    for meal in meal_types:
        if meal in text.lower():
            entities['meal_type'] = meal
            break
    
    for food in food_types:
        if food in text.lower():
            entities['food_type'] = food
            break
    
    for person in person_types:
        if person in text.lower():
            entities['person_type'] = person
            break
    
    for diet in diet_types:
        if diet in text.lower():
            entities['diet_type'] = diet
            break
    
    for disease in disease_types:
        if disease in text.lower():
            entities['disease_type'] = disease
            break
    
    return entities

# Entity extraction function
def extract_entities(text):
    bert_entities = ner_pipeline(text)
    rule_entities = rule_based_entity_extraction(text)
    
    entities = {
        'meal_type': rule_entities['meal_type'],
        'food_type': rule_entities['food_type'],
        'person_type': rule_entities['person_type'],
        'diet_type': rule_entities['diet_type'],
        'disease_type': rule_entities['disease_type'],
    }
    
    return entities

# Sentence embedding model
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
def get_sentence_embedding(text, model):
    return model.encode(text)

df_normal['embedding'] = df_normal['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))
df_diseased['embedding'] = df_diseased['features'].apply(lambda x: get_sentence_embedding(x, sentence_model))

# Food recommendation function
def recommend_food_based_on_entities(entities):
    if entities['disease_type']:
        user_input = f"{entities['food_type']} {entities['meal_type']} {entities['disease_type']}"
        user_embedding = get_sentence_embedding(user_input, sentence_model)
        similarities = cosine_similarity([user_embedding], np.stack(df_diseased['embedding'].values))
        idx = np.argmax(similarities)
        return df_diseased['recommend'].iloc[idx]
    else:
        user_input = f"{entities['food_type']} {entities['meal_type']} {entities['person_type']} {entities['diet_type']}"
        user_embedding = get_sentence_embedding(user_input, sentence_model)
        similarities = cosine_similarity([user_embedding], np.stack(df_normal['embedding'].values))
        idx = np.argmax(similarities)
        return df_normal['recommend'].iloc[idx]

# Edamam API credentials
EDAMAM_APP_ID = '2a98c6a4'  # Replace with your Edamam APP ID
EDAMAM_APP_KEY = '75704984fc122dc3153ae7a943f3cb56'  # Replace with your Edamam APP KEY

# Function to get nutritional information from Edamam API
def get_nutritional_info(food_item):
    url = f"https://api.edamam.com/api/nutrition-data?app_id={EDAMAM_APP_ID}&app_key={EDAMAM_APP_KEY}&nutrition-type=logging&ingr={food_item}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to format nutritional information for output
def format_nutrition_info(nutrition_data):
    if nutrition_data:
        nutrients = nutrition_data.get('totalNutrients', {})
        formatted_info = []
        for nutrient, details in nutrients.items():
            formatted_info.append(f"{details['label']}: {details['quantity']} {details['unit']}")
        return "\n".join(formatted_info)
    return "Nutritional information not available."

# GUI class
class MealRecommendationApp:
    def __init__(self, master):
        self.master = master
        master.title("Meal Recommendation System")

        self.label = tk.Label(master, text="Describe your diet preferences:")
        self.label.pack()

        self.entry = tk.Entry(master, width=50)
        self.entry.pack()

        self.recommend_button = tk.Button(master, text="Get Food Recommendation", command=self.get_recommendation)
        self.recommend_button.pack()

        self.nutrition_button = tk.Button(master, text="Get Nutritional Analysis", command=self.get_nutritional_analysis)
        self.nutrition_button.pack()

    def get_recommendation(self):
        prompt = self.entry.get()
        extracted_entities = extract_entities(prompt)
        recommendation = recommend_food_based_on_entities(extracted_entities)
        messagebox.showinfo("Recommended Food", recommendation)

    def get_nutritional_analysis(self):
        food_item = self.entry.get()
        nutrition = get_nutritional_info(food_item)
        nutritional_analysis = format_nutrition_info(nutrition)
        messagebox.showinfo("Nutritional Analysis", nutritional_analysis)

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = MealRecommendationApp(root)
    root.mainloop()


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\VIKRAM\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C: